In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor,create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

from dotenv import load_dotenv
load_dotenv()

db= SQLDatabase.from_uri("sqlite:///inventory1.db")
db.get_usable_table_names()

llm= ChatOpenAI(model="gpt-4o")
dbtoolktit=SQLDatabaseToolkit(db=db,llm=llm)
tools = [PythonREPLTool()] + dbtoolktit.get_tools()

prompt= """
    you are agent specialized in inventory management.

    You can analyse the database to identify low stock products.
    For each low stock product you have calculate the quantity to be
     orders and update in purchase order table
     
"""

prompttemplate=ChatPromptTemplate(
    [
        ("system", prompt),
         ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

agent = create_tool_calling_agent(
            llm=llm,
            tools=tools,
            prompt=prompttemplate
        )

agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True

)

agent_executor.invoke({
    "input": """
Inventory Parameters:
  - Safety Stock = Average Daily Sales × 3 days
  - Target Stock = Average Daily Sales × 7 days
  - Reorder Point = Safety Stock Level
  - Reorder Quantity = Target Stock + Safety Stock - Current Stock
  - Reorder Trigger: Current Stock ≤ Reorder Point

   Perform all the  the following tasks and dont miss any task:
                1. Check for low stock products. Every product has a reorder threshold.
                2. For each low stock product:                   
                   - Calculate optimal reorder quantity. You can calculate using the above formulas
                3.for each low stock product , create a purchase order and insert into
                  purchase_orders table
                4. Provide a summary of all actions taken


When ever you want to do any calculation, do it by using python code.
Dont do calculations in sql code
Make sure that the python code is not having any syntax issues
    """
})








> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


products, purchase_orders, sales_transactions, schema_version
Invoking: `sql_db_schema` with `{'table_names': 'products, sales_transactions, purchase_orders'}`



CREATE TABLE products (
	product_id TEXT, 
	name TEXT NOT NULL, 
	current_stock INTEGER NOT NULL, 
	reorder_threshold INTEGER NOT NULL, 
	supplier_email TEXT NOT NULL, 
	unit_price REAL NOT NULL, 
	supplier_name TEXT NOT NULL, 
	PRIMARY KEY (product_id)
)

/*
3 rows from products table:
product_id	name	current_stock	reorder_threshold	supplier_email	unit_price	supplier_name
P001	Gaming Mouse	25	30	sivaprasad.valluru@gmail.com	45.99	Tech Peripherals Inc
P002	Mechanical Keyboard	15	20	zerotosiva@gmail.com	89.99	Tech Peripherals Inc
P003	Gaming Headset	35	25	sivaprasad.valluru@gmail.com	79.99	Audio Tech Ltd
*/


CREATE TABLE purchase_orders (
	po_id INTEGER, 
	product_id TEXT NOT NULL, 
	quantity INTEGER NOT NULL, 
	order_date DATETIME NOT NULL, 


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `{'query': '# Calculation of reorder quantities and purchase orders for low stock products\nfrom datetime import datetime\n\n# Product details: product_id, product_name, current_stock, reorder_threshold\nproducts_info = [\n    (\'P001\', \'Gaming Mouse\', 25, 30),\n    (\'P002\', \'Mechanical Keyboard\', 15, 20),\n    (\'P003\', \'Gaming Headset\', 35, 25)\n]\n\n# Sales data: product_id, total_quantity_sold, sales_days\nsales_info = [\n    (\'P001\', 473, 30),\n    (\'P002\', 461, 30),\n    (\'P003\', 463, 30)\n]\n\n# Prepare a dictionary for easier access\nsales_dict = {product_id: (total_quantity, sales_days) for product_id, total_quantity, sales_days in sales_info}\n\n# Initialize list for purchase orders\npurchase_orders = []\n\n# Calculate reorder quantities\nfor product_id, name, current_stock, reorder_threshold in products_info:\n    total_quantity, sales_days = sales_dict[product_id]\n    average_daily_sales = total_quantity / sales_days\n    safet

{'input': '\nInventory Parameters:\n  - Safety Stock = Average Daily Sales × 3 days\n  - Target Stock = Average Daily Sales × 7 days\n  - Reorder Point = Safety Stock Level\n  - Reorder Quantity = Target Stock + Safety Stock - Current Stock\n  - Reorder Trigger: Current Stock ≤ Reorder Point\n\n   Perform all the  the following tasks and dont miss any task:\n                1. Check for low stock products. Every product has a reorder threshold.\n                2. For each low stock product:                   \n                   - Calculate optimal reorder quantity. You can calculate using the above formulas\n                3.for each low stock product , create a purchase order and insert into\n                  purchase_orders table\n                4. Provide a summary of all actions taken\n\n\nWhen ever you want to do any calculation, do it by using python code.\nDont do calculations in sql code\nMake sure that the python code is not having any syntax issues\n    ',
 'output': '##